# Pushing data to a MongoDB database

In [2]:
import os, sys
import pymongo

sys.path.append('../')
import src.dataProcessing as dp

In [3]:
utils = dp.Utills() 

# connect to mongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
BLE = client["BLE"]

In [9]:

def pushToDB(path,target, include_frame=True):
    totalFiles = len(os.listdir(path+target))
    currentFile = 0
    for x in os.scandir(path+target):
        if os.path.splitext(x)[1] != '.iq':
            continue
        
        target_path = os.path.splitext(x)[0]+os.path.splitext(x)[1]
        IQdatas,tindx = utils.readFile(target_path,Fc=2440000000.0 - 173000)

        for frame_nr in range(IQdatas.LEN):
            metaData = IQdatas.getMetaData(frame_nr, include_frame=include_frame)
            # print(target.split('/')[2].split('_')[1])
            if metaData == 0 or metaData['frameDecode'] == [0]:
                try:
                    if metaData['frameDecode'] == [0] or metaData['lenFrame'] <3000:
                        print(metaData)
                except:
                    pass
                continue
            collection = BLE[target.split('/')[1]]
            if len(metaData['frameDecode']) >= 10 and IQdatas.isServer(frame_nr) == False:
                collection.update_one( metaData,{'$set':metaData} ,upsert=True)
        currentFile += 1
        print(currentFile/totalFiles*100,'%')


path = '../DataSet/'
# targets = ['SDR_1/onBody/iter_1/','SDR_2/onBody/iter_1/',
#           'SDR_1/onBody/iter_2/','SDR_2/onBody/iter_2/']
# for target in targets:
#     pushToDB(path,target)

targets = [ 'SDR_2/offBody/high-tx-PWR-9dbm/', #'SDR_1/offBody/high-tx-PWR-9dbm/',
          'SDR_1/offBody/low-tx-PWR-3dbm/','SDR_2/offBody/low-tx-PWR-3dbm/']
for target in targets:
    pushToDB(path,target)

File name has a correct format!


TypeError: 'Collection' object is not callable. If you meant to call the 'update' method on a 'Collection' object it is failing because no such method exists.